In [1]:
# -*- coding: utf-8 -*-
import sklearn.feature_extraction
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
import numpy as np
from scipy import spatial
import timeit
import ast
from scipy import spatial
import operator

rate_dict={}
rate_pair=[]
movie_titles=[]

with open('movie_titles.txt', 'r') as f:
    for line in f:
        movie_titles.append(line[:-1])

print(len(movie_titles))
count = 0
with open('rate.txt', 'r') as r:
    for line in r:
        list1 = []
        list1.append(movie_titles[count])
        line = line[:-1]
        if(line == "ERROR"):
            line = 0.00
        rating = float(line)
        list1.append(rating)
        rate_pair.append(list1)
        count+=1
        if count == 5944:
            break


sorted_pair = [l[0] for l in rate_pair if l[1] > 8 and l[1] < 10]


print("number of movies(8<rating<10): "+str(len(sorted_pair)))
print("\n")
print("Vectorizing...")
print("\n")


def morphy(review):
    stop = ["있다", "다는", "은데", "특히", "있었", "동안", "면서", "을까", "해하", "어떤", "한때", "어야", "듯이", "ㄴ다", 'Story', "cinepark",
            "co", "kr", "Review", "★★★★★", "★★★★", "★★★", "★★", "★", '있', '하', '것', '들', '그', '되', '수', '이', '보', '않',
            '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문',
            '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중',
            '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤',
            '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '개', '전', '들', '사실',
            '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']

    try:
        kkma = Kkma()
        morphs = kkma.morphs(review)
        morphs_complete = []
        for i in morphs:
            if i not in stop and len(i) > 1:
                morphs_complete.append(i)


    except TypeError:
        print("TypeError has ocurred")
    return morphs_complete


#feat은 sad, horror 같은 feature 유의어 사전
#mor는 사용할 morphs (리뷰)
def featureDetector(feat, mor):
    cnt = 0.0
    for w in mor:
        if w in feat:
            cnt+=1
            if w in importantWords:
                cnt+=1
            if w in unimportantWords:
                cnt-=0.5
    return cnt

def featureNormalization(feature):
    total = 0
    for i in feature:
        total += i

    if total == 0:
        print(movie_name+": total is zero")
        with open("0_total.txt", "a") as s:
            s.write(movie_name + "'s total is " + str(total) + "\n")
        total = 1
    for i in range(len(feature)):feature[i] /= total
    return feature

def featurizer(feat):
    featStr = ','.join(feat)
    feat = morphy(featStr)
    feat = set(feat)
    return feat

def featureVectorizer(morphs):
    featureVector = []

    for i in feature_titles:
        featureVector.append(featureDetector(globals()[i], morphs))

    # 정규화
    featureNormalization(featureVector)

    return featureVector
sad = {'눈물나','사고', '사망','고독','자살', '알코올', '신파', '상처', '치유', '슬프','초라','비극','통탄', '애석', '원통', '비참', '애통', '애처로', '우울', '아프', '서럽', '침통', '비통', '비탄', '애틋', '서글프', '원통', '섧', '눈물', '침울', '무력', '버림받', '그림자', '그늘', '걱정', '근심','참혹', '참상', '시한부', '이별', '감성적', '싹트'}

horror = {'좀비', '감염', '불사', '숙주', '앨리스', '하이브', '전염', '호러', '멸망', '데드', '사일런트', '바이러스', '바이오하자드', '백신', '라쿤시티', '발병지', '재난', '번식', 'WTO','무섭', '목숨', '오싹', '긴장', '추격', '목숨','오싹', '시체', '잡아먹', '엽기', '정체불명', '자살', '고문','살해', '괴담', '살점', '끔찍하', '찌푸리', '질색', '죽이려','공포', '불안', '경악', '두려', '긴장', '숨통', '호러', '무서', '겁','주저', '오금', '가위', '악몽', '불안전', '위협', '살인', '살벌', '소름', '귀신', '괴물', '살인마', '실종', '흉가', '잔인','잔혹', '참혹', '참상','고어', '하드코어', '끔찍', '징그러', '흉하', '소름', '역겨', '벌레', '썩은', '흉측', '살인', '쏘우', '복수', '구역질', '토', '피', '살인','두려움'}

action = {'흥미진진','재미있','재미','재밌','악당', '미션', '스파이', '간첩', '007', '경찰','대역', '스턴트', '액션', '대규모', '세트장', '헐리우드', '훈련' ,'격투','대결','싸우','할리우드','사냥감','죽이','총격전','경찰관','마블','인도자','구출','악당', 'DC', 'Marvel'}

family = {'쌍둥이',  '보호', '따뜻', '가정적','임신', '고양이', '키우던', '애정', '각별', '동거', '홀로', '고향', '자살', '친엄마', '힐링', '가족적', '우애', '형제', '자매', '남매', '교훈', '가문', '물려받', '노총각','고명딸','명절', '휴먼', '성탄절','크리스마스이브','남동생','성장','소년','소녀','수련','가족','일족','집','혈통','종족','가구','친척','혈연','관계','자손','가정','아들','딸', '아버지', '어머니','아빠','엄마','부부','남편','아내','결혼','러브','이혼','임신','불임', '할아버지', '할머니', '손자', '손녀', '손주', '이모', '삼촌', '조카', 'love', 'affair', '감동', '입양','고향','드라마','눈물','웃음'}

SF= {'흥미진진','재미있','재미','재밌','외계', '세기', '태양계', '종족', '로봇', '재난', '우주인', '산소','지구', '우주','미래','과학','외계인','행성', '블랙홀','가상','스타워즈','매트릭스','우주선','은하','천체'}

love = {'매혹','인연', '보호', '따뜻', '동거', '각별', '임신', '멜로', '운명적', '만남', '짝사랑', '감동','로맨스','사랑','연인','호감','소중','매력','로맨틱','러브','애정','첫눈','이끌리','감정','남녀','관계','남편','아내','친분','재회','첫만남','결혼','러브','동성','성적','이별','애원','섹스','마음', '첫사랑', '행복', '낭만적', '연애관','몸매','욕망','운명론','속마음','청혼','낭만'}

hero = {'흥미진진','재미있','재미','재밌','대규모', '세트장', '헐리우드','거대','화려','넘치','장엄','현란','압도적','아이맥스', '블록버스터', '특수', '스케일', '분장','효과', '활약', '파트너','정의', '종말', '영웅','용맹','용기','대담','불굴','씩씩','히어로','마블','디씨','캡틴','리더','지도자','인도자','구출','악당', 'DC', 'Marvel','악역'}

comedy = {'흥미진진','재미있','재미','재밌','배꼽','노총각', '유머러스하', '재미있', '코미디', '유머', '명절', '웃기', '재미있는', '웃으', '유쾌', '떠들썩', '즐거운','코미디언','웃음'}

queer = {'동성애', '게이', '레즈', '레즈비언', '트렌스젠더', '퀴어', '동성'}

feminism = {'여권','여성','페미니즘', '양성', '페미','평등'}

time_travel = {'흥미진진','재미있','재미','재밌','타임', '돌아가', '퓨처', '타임머신', '시간여행', '미래','백투더퓨처'}

school = {'성적','청춘','성장', '명량', '고딩', '고등학생', '고등학교', '학교', '성장통', '성장기', '선생님', '학생', '교생', '교복', '사춘기', '짝사랑', '학원물', '청춘', '첫사랑','대학','청년'}

kid = {'아동용', '공주','애니', '극장판', '애니메이션', '캐릭터', '성우', '더빙', '어린이', '초등학생', '만화', 'TV시리즈','디즈니'}

docu = {'박근', '진지', '다큐','노무', '다큐멘터리', '실화','현실','기록','메세지','정치','일대기'}

disaster = {'대피', '환경', '오염', '온난','사고', '대책', '안보', '감염', '좀비', '바이러스', '사살', '시신', '생존자', '살아남', '목숨', '죽음', '희생', '지구', '진상','재난', '재해', '구조', '재앙', '멸망', '폭풍', '침몰', '전쟁', '대피'}

political = {'박근', '노무', '혁명', '정부', '시위', '투표', '정권', '퇴진', '정치', '노무현', '공화국', '민주주의', '독재', '대통령', '인권', '참정권', '운동', '노예','KKK','부조리','윤리적','정치적','메세지','역사','시대'}

war = {'병사', '무술', '침략', '명나라', '국방장관', '군부', '안보', '공산당', '대책', '미사일', '기지', '소련', '미군', '공군', '해군', '전투', '전우애', '전선', '휴전', '대대장', '북진', '남하', '부상병', '탱크', '폭탄', '냉전', '아군', '적군', 'GP', '초소', '전쟁', '소대원', '이병', '일병', '상병', '병장', '북한', '총', '칼', '공습','공성','총','작전','군대'}

thriller = {'흥미진진','재미있','재미','재밌','무장', '강도', '긴장감', '추격전','납치','복수극','인신매매', '쫓겨나', '어둠', '죽이', '서스펜스', '박진','무섭', '스릴러','폭력','피해자'}

crime = {'재미있','재미','재밌','누아르', '카지노', '밀항','밀입국','무장', '강도', '수사물', 'FBI','납치', '추격전', '사기', '죽이','인신매매','조직', '고스톱', '도박', '도박장','살인', '청부', '욕망', '파멸', '추락', '알코올', '중독', '심문', '강력계', '범죄', '마약', '도둑', '조폭', '형사', '경찰관','증거','기소','단속','불법','격투','범죄','마약','수사망','폭력','피해자','사건','출소','복역'}

twist = {'재미있','재미','재밌','반전', '반전있','스릴'}

music= {'교향곡','악장','왈츠','피아노','선율','재즈','연주','밴드','뮤지컬','음악','리드미컬하','리드미컬','멜로디','공연','무대','악기','조율','대중가요','노래','건반','기타','안무','뮤직비디오','희극','밴드','OST','ost','춤','댄스','무대'}

genius={'재미있','재미','재밌','추리', '두뇌','게임','베팅','배팅','설계','경기','대학','거물','카드','도박','천재','아이비','심리'}

fantasy={'흥미진진','재미있','재미','재밌','반지', '요술', '지팡이', '마법', '빗자루', '엘프', '제왕', '오크', '주문', '마법사', '판타지', '드래곤', '롤링', '워프', '해리','환상','아라곤','난쟁이','간달프','원정대','드워프','사우론','괴물','해리','포터','마법','유니콘','요정','CG','성물','왕자'}

sports = {'흥미진진','재미있','재미','재밌','선수', '대결', '스포츠', '운동', '농구', '축구', '하키', '역경', '우승', '야구', '투수', '투구', '공격수', '수비수', '승부'}

religious = {'천사', '악마', '미사', '수녀', '성당', '세례', '십자가', '예수', '사제', '바티칸', '교황', '영혼', '종교적', '목사','교회','참배','종교인','선교사','피어스','템플','하느님','스님','미카엘','진리','큰스님','불교','계도','부처','성경','구절','순례'}

feature_titles = ['sad','horror','action','family','SF','love','hero','comedy','queer','feminism','time_travel','school','kid','docu','disaster','political','war','thriller','crime','twist','music','genius','fantasy','sports','religious']


##가중치주기:
importantWords = {'뮤지컬','좀비','공포','슬프','눈물','호러','귀신','괴물','살인','액션','할리우드','헐리우드','마블','DC','Marvel','가족적','명절','멜로','섹스','연애','결혼','영웅','히어로','웃기','웃으','코미디','코메디','페미니즘','동성애','게이','레즈','타임머신','시간여행','학교','성장기','첫사랑','아동용','애니','극장판','애니메이션','더빙','만화','디즈니','재난','재앙','민주주의','역사','병사','아군','전쟁','북한','긴장감','서스펜스','박진','스릴러','범죄','조폭','형사','폭력','반전','반전있','음악','선율','밴드','노래','공연','추리','두뇌','천재','마법','마법사','판타지','스포츠','선수','종교적','하나님','하느님','스님','불교','우주'}
unimportantWords = {'흥미진진','재미있','재미'}


#stops = ["★★★★★", "★★★★", "★★★", "★★", "★", '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


#stopword: 2글자 이상 단어 중 필요 없는 단어 계속 추가해서 필터링!



from os import listdir
from os.path import isfile, join

files = [f for f in listdir('C:/Users/HKH/Documents/카카오톡 받은 파일/알티캐~1/데모용 소스코드(final)/cine21') if isfile(join('C:/Users/HKH/Documents/카카오톡 받은 파일/알티캐~1/데모용 소스코드(final)/cine21', f))]
movie_titles=[]
for i in files:
    tmp= i.split('.txt')
    movie_titles.append(tmp[0])

vector_list = []
count = 0
movie_titles=[]
import os,glob
f = open('movie_titles(ver2).txt', 'r')
lines = f.readlines()
os.chdir('C:/Users/HKH/Documents/카카오톡 받은 파일/알티캐~1/데모용 소스코드(final)/cine21')
count=1
print('lets start!')
for line in lines:
    line=line.replace('\n','')
    filename=line+'.txt'
    with open(filename, 'r', encoding='UTF8') as f:
        review1 = f.read()
        morphs1 = morphy(review1)
        if (len(morphs1) <= 150):
            continue
        feature1 = featureVectorizer(morphs1)

        vector_list.append(feature1)
        movie_titles.append(line)
        with open("vec.txt", "a") as v:
            v.write(str(feature1) + "\n")
        with open("mov.txt", "a") as m:
            m.write(line + "\n")
        print(str(count)+':'+filename)
        count += 1


5944
number of movies(8<rating<10): 2005


Vectorizing...


lets start!
1:A-특공대(2010).txt
2:A.I.(2001).txt
3:BB프로젝트(2006).txt
4:B급 며느리(2017).txt
5:D-13(2000).txt
6:K-19 위도우메이커(2002).txt
7:Mr.스타벅(2011).txt
8:Mr.히치 당신을 위한 데이트 코치(2005).txt
9:P.S 아이러브유(2007).txt
10:REC 알이씨(2009).txt
11:가디안(2000).txt
12:가디언의 전설(2010).txt
13:가디언즈 오브 갤럭시 VOL. 2(2017).txt
14:가디언즈 오브 갤럭시(2014).txt
15:가려진 시간(2015).txt
16:가부와 메이이야기(2005).txt
17:가을 소나타(1978).txt
18:가장 따뜻한 색, 블루(2013).txt
19:가족시네마(2012).txt
20:가족의 나라(2011).txt
21:각설탕(2006).txt
22:간츠(2011).txt
23:갈매기(2018).txt
24:감바의 대모험(2015).txt
25:갓파 쿠와 여름방학을(2007).txt
26:강을 건너는 사람들(2006).txt
27:강철비(2017).txt
28:강철중 공공의 적 1-1(2008).txt
29:개들의 섬(2018).txt
30:개를 훔치는 완벽한 방법(2014).txt
31:개와 고양이(2004).txt
32:갬빗(2012).txt
33:거북이 달린다(2008).txt
34:거북이도 난다(2004).txt
35:거울나라의 앨리스(2016).txt
36:건축학개론(2012).txt
37:걸 온 더 브릿지(1999).txt
38:걸스 라이프(2003).txt
39:걸스 온 탑(2001).txt
40:걸어도 걸어도(2008).txt
41:걸즈 앤 판처 극장판(2015).txt
42:검은 사제들(2015).txt
43:겁쟁이 페달 더 무비(2015).txt
44:겜블(1999).t

358:두 번의 결혼식과 한 번의 장례식(2012).txt
359:두갈  마법의 회전목마(2005).txt
360:두레소리(2012).txt
361:드라큘라 전설의 시작(2014).txt
362:드래곤 길들이기 2(2014).txt
363:드래곤 길들이기(2010).txt
364:드래곤볼Z 신들의 전쟁(2013).txt
365:드럼라인(2002).txt
366:드레스메이커(2015).txt
367:드롭박스(2015).txt
368:드림걸즈(2006).txt
369:드림업(2009).txt
370:드림캐쳐(2003).txt
371:드림하우스(1998).txt
372:디 아워스(2002).txt
373:디센던트(2011).txt
374:디스 민즈 워(2012).txt
375:디스커넥트(2012).txt
376:디스트릭트 9(2009).txt
377:디어 한나(2011).txt
378:디어평양(2006).txt
379:디올 앤 아이(2014).txt
380:디태치먼트(2011).txt
381:디트로이트(2017).txt
382:디파이언스(2008).txt
383:딜쿠샤(2015).txt
384:딥워터 호라이즌(2016).txt
385:땅의 여자(2009).txt
386:땡스 포 쉐어링(2012).txt
387:땡큐 포 유어 서비스(2017).txt
388:땡큐, 대디(2013).txt
389:땡큐, 마스터 킴(2008).txt
390:또 하나의 약속(2013).txt
391:똥파리(2008).txt
392:뚜르 내 생애 최고의 49일(2016).txt
393:뜨거운 녀석들(2007).txt
394:뜨거운 안녕(2012).txt
395:라디오스타(2006).txt
396:라따뚜이(2007).txt
397:라라랜드(2016).txt
398:라벤다(2000).txt
399:라벤더의 연인들(2004).txt
400:라비앙 로즈(2007).txt
401:라스베가스를 떠나며(1995).txt
402:라스베가스에서만 생길 수 있는일(2008).txt
403:라스트 러브 인 뉴욕

722:벌이 날다(1998).txt
723:범블비(2018).txt
724:범죄도시(2017).txt
725:범죄와의 전쟁  나쁜놈들 전성시대(2011).txt
726:범죄의 여왕(2015).txt
727:범죄의 재구성(2004).txt
728:법정스님의 의자(2011).txt
729:베로니카 게린(2003).txt
730:베로니카의 이중생활(1991).txt
731:베른의 기적(2003).txt
732:베스트 오퍼(2013).txt
733:베스트 키드(2010).txt
734:베이비 드라이버(2017).txt
735:베일리 어게인(2017).txt
736:베일을 쓴 소녀(2013).txt
737:베테랑(2015).txt
738:벤다 빌릴리(2010).txt
739:벤자민 버튼의 시간은 거꾸로 간다(2008).txt
740:벤허(2016).txt
741:벨과 세바스찬(2013).txt
742:벨벳 골드마인(1998).txt
743:벨빌의 세 쌍둥이(2003).txt
744:벼랑 위의 포뇨(2008).txt
745:변호인(2013).txt
746:별별 이야기(2005).txt
747:별이 빛나는 밤(2011).txt
748:보글보글 스폰지밥(2004).txt
749:보루토 -나루토 더 무비-(2015).txt
750:보리밭을 흔드는 바람(2006).txt
751:보리울의 여름(2003).txt
752:보물성(2002).txt
753:보스 베이비(2017).txt
754:보이 A(2007).txt
755:보이 걸 씽(2006).txt
756:보헤미안 랩소디(2018).txt
757:본 슈프리머시(2004).txt
758:본 아이덴티티(2002).txt
759:본 얼티메이텀(2007).txt
760:볼링 포 콜럼바인(2002).txt
761:볼쇼이 바빌론(2015).txt
762:봄(2014).txt
763:봄날은 간다(2001).txt
764:봄날의 곰을 좋아하세요(2003).txt
765:봄여름가을겨울 그리고 봄(2003).txt
766:부당거래(2010).tx

1088:아귀레, 신의 분노(1972).txt
1089:아기기린 자라파(2012).txt
1090:아기돼지 3형제와 쿵푸랜드(2017).txt
1091:아기배달부 스토크(2016).txt
1092:아내업고 달리기(2015).txt
1093:아뉴스 데이(2016).txt
1094:아는 여자(2004).txt
1095:아더 크리스마스(2011).txt
1096:아더와 미니모이  제1탄 비밀 원정대의 출정(2008).txt
1097:아들에게 가는 길(2017).txt
1098:아들의 방(2001).txt
1099:아따맘마 - 극장판(2003).txt
1100:아라가미(2002).txt
1101:아라한 장풍대작전(2004).txt
1102:아르고(2012).txt
1103:아마존의 눈물(2010).txt
1104:아메리칸 갱스터(2007).txt
1105:아메리칸 뷰티(1999).txt
1106:아메리칸 스나이퍼(2014).txt
1107:아메리칸 파이  19금 동창회(2012).txt
1108:아모레스 페로스(2000).txt
1109:아무도 머물지 않았다(2013).txt
1110:아무르(2012).txt
1111:아미티빌 호러(2005).txt
1112:아버지와 마리와 나(2007).txt
1113:아버지와 이토씨(2016).txt
1114:아버지의 이메일(2012).txt
1115:아부지(2009).txt
1116:아쉬람(2005).txt
1117:아스라이(2007).txt
1118:아스트로 보이 - 아톰의 귀환(2009).txt
1119:아워 이디엇 브라더(2011).txt
1120:아유레디(2013).txt
1121:아이 노우 유 노우(2008).txt
1122:아이 앰 히스 레저(2017).txt
1123:아이 엠 샘(2001).txt
1124:아이 오브 더 스톰(2011).txt
1125:아이 인 더 스카이(2015).txt
1126:아이 캔 스피크(2017).txt
1127:아이 킬드 마이 마더(2009).txt
1128:아이, 로봇(2004).txt

1434:이탈리아 횡단밴드(2010).txt
1435:이탈리안 잡(2003).txt
1436:이터널 선샤인(2004).txt
1437:이티(1982).txt
1438:이파네마 소년(2010).txt
1439:인 더 하우스(2012).txt
1440:인 디 아일(2018).txt
1441:인 디스 월드(2002).txt
1442:인게이지먼트(2004).txt
1443:인디애니박스  셀마의 단백질 커피(2008).txt
1444:인사동 스캔들(2008).txt
1445:인사이더(1999).txt
1446:인사이드 딥 스로트(2005).txt
1447:인사이드 르윈(2013).txt
1448:인사이드 아웃(2015).txt
1449:인사이드 잡(2010).txt
1450:인생을 애니메이션처럼(2017).txt
1451:인셉션(2010).txt
1452:인어베러월드(2010).txt
1453:인크레더블 2(2018).txt
1454:인크레더블 헐크(2008).txt
1455:인크레더블(2004).txt
1456:인터뷰(2000).txt
1457:인터뷰(2007).txt
1458:인형의 집으로 오세요(1995).txt
1459:일급기밀(2015).txt
1460:일대종사(2012).txt
1461:일루미나타(1998).txt
1462:일만명의 성자들(2015).txt
1463:일주일간 친구(2017).txt
1464:잉글리쉬 브라이드(2001).txt
1465:잉여들의 히치하이킹(2013).txt
1466:잊혀진 가방(2010).txt
1467:잊혀진 꿈의 동굴(2010).txt
1468:자, 이제 댄스타임(2013).txt
1469:자반고등어(2000).txt
1470:자백(2016).txt
1471:자유로운 세계(2007).txt
1472:자토이치(2003).txt
1473:작은 형(2014).txt
1474:작전명 발키리(2008).txt
1475:잠깐만 회사 좀 관두고 올게(2017).txt
1476:잠베지아  신비한 나무섬의 비밀(2012).txt
1477:잠

1787:페이스 메이커(2011).txt
1788:페이스 오브 러브(2013).txt
1789:페이퍼 하트(2010).txt
1790:페인티드 베일(2006).txt
1791:펭귄 위대한 모험(2005).txt
1792:포미니츠(2006).txt
1793:포켓 몬스터 2 - 루기아의 탄생(1999).txt
1794:포켓몬 더 무비 XY 후파 광륜의 초마신(2015).txt
1795:포화속으로(2009).txt
1796:폭력의 법칙 나쁜 피 두 번째 이야기(2016).txt
1797:폭스캐처(2014).txt
1798:폰부스(2002).txt
1799:폴 다이어리(2010).txt
1800:폴라 익스프레스(2004).txt
1801:폴라로이드(2015).txt
1802:폴레트의 수상한 베이커리(2012).txt
1803:폴리스 스토리 2014(2013).txt
1804:폼포코 너구리 대작전(1994).txt
1805:푸른 노을(2017).txt
1806:품행제로(2002).txt
1807:풍뎅이뎅이(2017).txt
1808:퓨리(2014).txt
1809:프라임 러브(2005).txt
1810:프란츠(2016).txt
1811:프란치스코(2015).txt
1812:프랑켄위니(2012).txt
1813:프레스티지(2006).txt
1814:프렌즈  몬스터 섬의 비밀 (2011).txt
1815:프렌즈  하얀 거짓말(2010).txt
1816:프렌치 커넥션 마약수사(2014).txt
1817:프로듀서스(2005).txt
1818:프로스트 vs 닉슨(2008).txt
1819:프로젝트 님(2011).txt
1820:프로포즈 데이(2010).txt
1821:프리다(2002).txt
1822:프리다의 그해 여름(2017).txt
1823:프리즈 프레임(2004).txt
1824:프리즈너스(2013).txt
1825:프리키 프라이데이(2003).txt
1826:프린세스 다이어리(2001).txt
1827:프린스 앤 프린세스(1999).txt
1828:플라워 쇼(2015)